<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
# текст запроса
query_3_1 = f'''
select count(*)
from public.vacancies
'''

In [5]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df.rename(columns={'count':'vacancies'},inplace=True)
df

C:\Users\e.kuryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,vacancies
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
# текст запроса
query_3_2 = f"""
select count(*)
from public.employers
"""

In [7]:
# результат запроса
df['employers'] = pd.read_sql_query(query_3_2,connection)
df

C:\Users\e.kuryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,vacancies,employers
0,49197,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
# текст запроса
query_3_3 = f"""
select count(*)
from public.areas
"""

In [9]:
# результат запроса
df['areas'] = pd.read_sql_query(query_3_3,connection)
df

C:\Users\e.kuryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,vacancies,employers,areas
0,49197,23501,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
# текст запроса
query_3_4 = f"""
select count(*)
from public.industries
"""

In [11]:
# результат запроса
df['industries'] = pd.read_sql_query(query_3_4,connection)
df

C:\Users\e.kuryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,vacancies,employers,areas,industries
0,49197,23501,1362,294


***

In [12]:
df['vacan_per_empl'] = df.vacancies/df.employers
df['empl_per_area'] = df.employers/df.areas
df['empl_per_industries'] = df.employers/df.industries
df['vacan_per_area'] = df.vacancies/df.areas
df


,vacancies,employers,areas,industries,vacan_per_empl,empl_per_area,empl_per_industries,vacan_per_area
0,49197,23501,1362,294,2.0934,17.254772,79.935374,36.121145


### выводы по предварительному анализу данных
При поверхностном взгляде на данные можно сделать следующие выводы:
1. В представленной базе 49197 вакансий от 23501 работодателя, распологающихся в 1362 городах и разделенных на 294 сферы деятельности. 
2. В среднем на одного работодателя приходиться 2 вакансии. 
3. В среднем в одном городе распологается 17 работодателей и открыто 36 вакансий.
4. Среднее значения по количеству работодателей в каждой сфере деятельности 79.

Для корректного понимания данных необходим более глубокий анализ, т.к. средние значения скорее всего плохо отображают реальность.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [13]:
# текст запроса
query_4_1 = f"""
select 
a.name area, 
count(*) cnt
from public.vacancies v
left join public.areas a on v.area_id = a.id
group by 1
order by 2 desc
"""

In [14]:
# результат запроса
df_vac_in_area = pd.read_sql_query(query_4_1,connection)
df_vac_in_area['percentile'] = df_vac_in_area.cnt/df_vac_in_area.cnt.sum()
df_vac_in_area

C:\Users\e.kuryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,area,cnt,percentile
0,Москва,5333,0.108401
1,Санкт-Петербург,2851,0.057951
2,Минск,2112,0.042929
3,Новосибирск,2006,0.040775
4,Алматы,1892,0.038458
...,...,...,...
764,Тарко-Сале,1,0.000020
765,Новоаннинский,1,0.000020
766,Бирск,1,0.000020
767,Сасово,1,0.000020


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [15]:
# текст запроса
query_4_2 = f"""
select count(*) any_salary_not_blank_count
from public.vacancies
where salary_to is not null or salary_from is not null
"""

In [16]:
# результат запроса
df_detailed = pd.read_sql_query(query_4_2,connection)
df_detailed

C:\Users\e.kuryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,any_salary_not_blank_count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [17]:
# текст запроса
query_4_3 = f"""
select 
round(avg(salary_from),0) avg_salary_from, 
round(avg(salary_to),0) avg_salary_to
from public.vacancies
"""

In [18]:
# результат запроса
df_detailed= pd.concat([df_detailed,pd.read_sql_query(query_4_3,connection)],axis=1)
df_detailed

C:\Users\e.kuryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,any_salary_not_blank_count,avg_salary_from,avg_salary_to
0,24073,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [19]:
# текст запроса
query_4_4 = f"""
select 
schedule, 
employment, 
count(*)
from public.vacancies
group by 1,2
order by 3 desc
"""

In [20]:
# результат запроса
df_schedule = pd.read_sql_query(query_4_4,connection)
df_schedule['percentile'] = df_schedule['count']/df_schedule['count'].sum()
df_schedule

C:\Users\e.kuryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,schedule,employment,count,percentile
0,Полный день,Полная занятость,35367,0.718885
1,Удаленная работа,Полная занятость,7802,0.158587
2,Гибкий график,Полная занятость,1593,0.032380
3,Удаленная работа,Частичная занятость,1312,0.026668
4,Сменный график,Полная занятость,940,0.019107
5,Полный день,Стажировка,569,0.011566
6,Вахтовый метод,Полная занятость,367,0.007460
7,Полный день,Частичная занятость,347,0.007053
8,Гибкий график,Частичная занятость,312,0.006342
9,Полный день,Проектная работа,141,0.002866


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [21]:
# текст запроса
query_4_5 = f"""
select 
experience, 
count(*), 
round(avg(salary_from),0) avg_salary_from, 
round(avg(salary_to),0) avg_salary_to
from public.vacancies
group by 1
order by 2 
"""

In [22]:
# результат запроса
df_exp = pd.read_sql_query(query_4_5,connection)
df_exp['percentiel'] = df_exp['count']/df_exp['count'].sum()
df_exp

C:\Users\e.kuryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,experience,count,avg_salary_from,avg_salary_to,percentiel
0,Более 6 лет,1337,150156.0,197819.0,0.027176
1,Нет опыта,7197,34699.0,56990.0,0.146289
2,От 3 до 6 лет,14511,112544.0,171974.0,0.294957
3,От 1 года до 3 лет,26152,64456.0,97436.0,0.531577


***

In [23]:
df_detailed['num_of_area_with_most_vac'] = df_vac_in_area[df_vac_in_area['cnt']/df_vac_in_area['cnt'].sum()>0.02]['cnt'].count()
df_detailed['num_of_vac_in_top_areas'] = df_vac_in_area[df_vac_in_area['cnt']/df_vac_in_area['cnt'].sum()>0.02]['cnt'].sum()
df_detailed['percent_of_vac_in_top_areas'] = df_vac_in_area[df_vac_in_area['cnt']/df_vac_in_area['cnt'].sum()>0.02]['cnt'].sum()/df_vac_in_area.cnt.sum()
df_detailed

,any_salary_not_blank_count,avg_salary_from,avg_salary_to,num_of_area_with_most_vac,num_of_vac_in_top_areas,percent_of_vac_in_top_areas
0,24073,71065.0,110537.0,12,23616,0.480029


### Выводы по детальному анализу вакансий
Как и предполагалось ранее, средние значения не дают корректной информации по данным. Так, наибольшее число вакансий представлено в Москве - практически 11% от всех вакансий. При этом на 12 городов (из 769) приходится 48% всех вакансий (23616 из 49197). 

Большинство вакансий открыто на полную занятось (71%) при этом в базе присутсвует 15% вакансий с удаленной занятостью на полный рабочий день.

51% открытых вакансий требуют опыта работы от 1 до 3 лет, с требование к опыту более 6 лет открыто всего 2.7% вакансий

В половине вакансий указана предлагаемая зарплата (в 24073 из 49197) при этом иходя из данных в базе видна прямая зависимость между опытом работы и предлагаемой зарплатной вилкой (что логично). Так для вакансий без опыта работы вилка в среднем составляет от 34699 до 56990, для вакансий с опытом работы более 6 лет вилка в среднем составляет от 150156 до 197819.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [24]:
# текст запроса
query_5_1 = f"""
(select 
e.name empl_name, 
count(*) vacan_num
from public.vacancies v
left join public.employers e on v.employer_id = e.id
group by 1
order by 2 desc
limit 1
)
union all
(
select 
e.name empl_name, 
count(*) vacan_num
from public.vacancies v
left join public.employers e on v.employer_id = e.id
group by 1
order by 2 desc
offset 4
limit 1
)
"""

In [25]:
# результат запроса
top_vacancies = pd.read_sql_query(query_5_1,connection)
top_vacancies

C:\Users\e.kuryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,empl_name,vacan_num
0,Яндекс,1933
1,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [26]:
# текст запроса
query_5_2 = f"""
with t1 as 
(select 
a.id, 
a.name,
count(e.name) num_of_empl
from public.areas a
left join public.employers e on a.id=e.area 
group by 1,2
),

t2 as 
(select 
a.id, 
a.name,
count(v.name) num_of_vac
from public.areas a
left join public.vacancies v on v.area_id = a.id
group by 1,2
)

select t1.name city,
t1.num_of_empl, 
t2.num_of_vac
from t1
left join t2 on t1.id=t2.id 
order by 3,2 desc 
"""

In [27]:
# результат запроса
region_emp_vacancies = pd.read_sql_query(query_5_2,connection)
region_emp_vacancies

C:\Users\e.kuryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,city,num_of_empl,num_of_vac
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Ростовская область,18,0
...,...,...,...
1357,Алматы,721,1892
1358,Новосибирск,573,2006
1359,Минск,1115,2112
1360,Санкт-Петербург,2217,2851


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [28]:
# текст запроса
query_5_3 = f"""
select 
e.name company, 
count(distinct(a.id)) num_of_regions
from public.vacancies v
left join public.employers e on v.employer_id = e.id
left join public.areas a on v.area_id = a.id
group by e.id
order by 2 desc
"""

In [29]:
# результат запроса
empl_areas = pd.read_sql_query(query_5_3,connection)
empl_areas

C:\Users\e.kuryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,company,num_of_regions
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14901,НПП Авиатрон,1
14902,Центр дистанционных торгов,1
14903,Городские Телекоммуникационные Системы,1
14904,"Введенский, Отель",1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [30]:
# текст запроса
query_5_4 = f"""
select count(e.id) num_empl_without_industries
from public.employers e
left join public.employers_industries ei on e.id = ei.employer_id
where ei.industry_id is null
""" 

In [31]:
# результат запроса
num_empl_without_industry = pd.read_sql_query(query_5_4,connection)
num_empl_without_industry

C:\Users\e.kuryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,num_empl_without_industries
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [32]:
# текст запроса
query_5_5 = f"""
select e.name company_name, count(ei.industry_id) num_of_industries
from public.employers e
left join public.employers_industries ei on e.id = ei.employer_id
group by e.id
having count(ei.industry_id) = 4
order by 1
offset 2
limit 1
""" 

In [33]:
# результат запроса
third_with_four_indust= pd.read_sql_query(query_5_5,connection)
third_with_four_indust

C:\Users\e.kuryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,company_name,num_of_industries
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [34]:
# текст запроса
query_5_6 = f"""
select count(e.name) empl_with_it_vacancies
from public.employers e
left join public.employers_industries ei on e.id = ei.employer_id
left join public.industries i on ei.industry_id = i.id
where i.name like 'Разработка программного обеспечения'
""" 

In [35]:
# результат запроса
num_of_empl_with_it_indust = pd.read_sql_query(query_5_6,connection)
num_of_empl_with_it_indust

C:\Users\e.kuryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,empl_with_it_vacancies
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [36]:
# код для получения списка городов-милионников
import requests as req
from bs4 import BeautifulSoup
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
resp = req.get(url)
page = BeautifulSoup(resp.text,'html.parser')
table = page.find('table',class_='standard sortable')
df = pd.read_html(str(table))
df = pd.concat(df)
df['Город'][13] = 'Воронеж'
df['Город']

C:\Users\e.kuryan\AppData\Local\Temp\ipykernel_10176\2981509396.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Город'][13] = 'Воронеж'


0              Москва
1     Санкт-Петербург
2         Новосибирск
3        Екатеринбург
4              Казань
5     Нижний Новгород
6           Челябинск
7          Красноярск
8              Самара
9                 Уфа
10     Ростов-на-Дону
11               Омск
12          Краснодар
13            Воронеж
14              Пермь
15          Волгоград
Name: Город, dtype: object

In [37]:
million_plus_cities = tuple(df['Город'])
million_plus_cities

('Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Челябинск',
 'Красноярск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Омск',
 'Краснодар',
 'Воронеж',
 'Пермь',
 'Волгоград')

In [38]:
# текст запроса
query_5_7 = f"""
with table1 as 
(
select a.name city, 
count(a.name) num_of_vacancies
from public.employers e
left join public.vacancies v on e.id = v.employer_id
left join public.areas a on v.area_id=a.id
where e.name like 'Яндекс' and a.name in {million_plus_cities}
group by 1
order by 2 desc
)


select *
from table1

union all

select 'Total', sum(num_of_vacancies)
from table1

"""


In [39]:
yandex_vac_in_mill_areas = pd.read_sql_query(query_5_7,connection)
yandex_vac_in_mill_areas


C:\Users\e.kuryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,city,num_of_vacancies
0,Москва,54.0
1,Санкт-Петербург,42.0
2,Екатеринбург,39.0
3,Нижний Новгород,36.0
4,Новосибирск,35.0
5,Воронеж,32.0
6,Краснодар,30.0
7,Самара,26.0
8,Уфа,26.0
9,Казань,25.0


***

In [40]:
# выводы по анализу работодателей
display(yandex_vac_in_mill_areas)
display(num_of_empl_with_it_indust)
display(third_with_four_indust)
display(num_empl_without_industry)
display(empl_areas)
display(region_emp_vacancies)
display(top_vacancies)

,city,num_of_vacancies
0,Москва,54.0
1,Санкт-Петербург,42.0
2,Екатеринбург,39.0
3,Нижний Новгород,36.0
4,Новосибирск,35.0
5,Воронеж,32.0
6,Краснодар,30.0
7,Самара,26.0
8,Уфа,26.0
9,Казань,25.0


,empl_with_it_vacancies
0,3553


,company_name,num_of_industries
0,2ГИС,4


,num_empl_without_industries
0,8419


,company,num_of_regions
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14901,НПП Авиатрон,1
14902,Центр дистанционных торгов,1
14903,Городские Телекоммуникационные Системы,1
14904,"Введенский, Отель",1


,city,num_of_empl,num_of_vac
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Ростовская область,18,0
...,...,...,...
1357,Алматы,721,1892
1358,Новосибирск,573,2006
1359,Минск,1115,2112
1360,Санкт-Петербург,2217,2851


,empl_name,vacan_num
0,Яндекс,1933
1,Газпром нефть,331


### выводы по анализу работодателей
Наибольшее количество работодателей находится в Москве, так же в этом городе открыто наибольшее количество вакансий. Отдельно вывев инфомрацию по вакансиям "Яндекса", получили ожидаемую информацию о том, что наибольшее количество вакансий открыто так же в Москве.


В базе данных 3553 компании предлагаю вакансии связанные с ИТ. При этом 8419 работодателей не указали сферу деятельности.


# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [41]:
# текст запроса
query_6_1 = f"""
select count(*) data_vac_count
from public.vacancies
where 
lower(name) like '%data%' 
or 
lower(name) like '%данн%'
"""

In [42]:
# результат запроса
df = pd.read_sql_query(query_6_1, connection)
df

C:\Users\e.kuryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_vac_count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [43]:
# текст запроса
query_6_2 = f"""
select count(*) num_of_vacan_jun_data
from public.vacancies
where 
(name ilike '%data scientist%' 
or 
name ilike '%data science%'
or 
name ilike '%исследователь данных%'
or
(name ilike '%ml%' and name not ilike '%html%')
or 
name ilike '%machine learning%'
or 
name ilike '%машинн%обучен%')
and
(name ilike '%junior%' 
or
experience = 'Нет опыта'
or
employment = 'Стажировка'
)
"""

In [44]:
# результат запроса
df['num_of_jun_data_vac'] = pd.read_sql_query(query_6_2,connection)
df

C:\Users\e.kuryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_vac_count,num_of_jun_data_vac
0,1771,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [45]:
# текст запроса
query_6_3 = f"""
select count(*) num_of_vacan_data_key_sql
from public.vacancies
where 
(name ilike '%data scientist%' 
or 
name ilike '%data science%'
or 
name ilike '%исследователь данных%'
or
(name like '%ML%' and name not like '%HTML%')
or 
name ilike '%machine learning%'
or 
name ilike '%машинн%обучен%')
and
(key_skills like '%SQL%' 
or 
key_skills like '%postgres%')
"""

In [46]:
# результат запроса
df['num_of_data_vacan_with_sql'] = pd.read_sql_query(query_6_3,connection)
df

C:\Users\e.kuryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_vac_count,num_of_jun_data_vac,num_of_data_vacan_with_sql
0,1771,51,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [47]:
# текст запроса
query_6_4 = f"""
select count(*) num_of_vacan_data_key_python
from public.vacancies
where 
(name ilike '%data scientist%' 
or 
name ilike '%data science%'
or 
name ilike '%исследователь данных%'
or
(name like '%ML%' and name not like '%HTML%')
or 
name ilike '%machine learning%'
or 
name ilike '%машинн%обучен%')
and
(key_skills ilike '%python%' 
)
"""

In [48]:
# результат запроса
df['num_of_data_vac_with_python'] = pd.read_sql_query(query_6_4,connection)
df

C:\Users\e.kuryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_vac_count,num_of_jun_data_vac,num_of_data_vacan_with_sql,num_of_data_vac_with_python
0,1771,51,201,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [49]:
# текст запроса
query_6_5 = f"""
select round(avg(length(key_skills) - length(replace(key_skills,'\t',''))+1),2) avg_skills_in_ds_vac
from public.vacancies
where 
(name ilike '%data scientist%' 
or 
name ilike '%data science%'
or 
name ilike '%исследователь данных%'
or
(name like '%ML%' and name not like '%HTML%')
or 
name ilike '%machine learning%'
or 
name ilike '%машинн%обучен%')
"""

In [50]:
# результат запроса
df['avg_num_of_skill_in_ds_vac'] = pd.read_sql_query(query_6_5,connection)
df

C:\Users\e.kuryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_vac_count,num_of_jun_data_vac,num_of_data_vacan_with_sql,num_of_data_vac_with_python,avg_num_of_skill_in_ds_vac
0,1771,51,201,351,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [51]:
# текст запроса
query_6_6 = f"""
with table1 as (select experience,
case when (salary_from is not null and salary_to is not null)
then
(salary_from +salary_to)/2
else
coalesce(salary_from, 0) + coalesce(salary_to, 0)
end as avg_salary
from public.vacancies
where 
(name ilike '%data scientist%' 
or 
name ilike '%data science%'
or 
name ilike '%исследователь данных%'
or
(name like '%ML%' and name not like '%HTML%')
or 
name ilike '%machine learning%'
or 
name ilike '%машинн%обучен%')
and (
salary_to is not null 
or
salary_from is not null  
))

select experience, round(avg(avg_salary),0) avg_salary
from table1
group by 1

"""

In [52]:
# результат запроса
df_avg_salary_for_exp = pd.read_sql_query(query_6_6,connection)
df_avg_salary_for_exp

C:\Users\e.kuryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,experience,avg_salary
0,От 3 до 6 лет,243115.0
1,От 1 года до 3 лет,139675.0
2,Нет опыта,74643.0


***

In [65]:
# выводы по предметному анализу
query_6_7 = f"""
select round(avg(length(key_skills) - length(replace(key_skills,'\t',''))+1),2) avg_skills_in_ds_vac
from public.vacancies
"""
df['avg_num_of_skill_in_all_vac'] = pd.read_sql_query(query_6_7,connection)
df

C:\Users\e.kuryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,data_vac_count,num_of_jun_data_vac,num_of_data_vacan_with_sql,num_of_data_vac_with_python,avg_num_of_skill_in_ds_vac,avg_num_of_skill_in_all_vac
0,1771,51,201,351,6.41,6.37


### Выводы по предметному анализу
В базе данных 1771 вакансия каким-либо образом связанные с данными. При этом для джунов есть 51 вакансия, 201 вакансия связана с знание SQL, 351 вакансия связана со знанием Python.

Согласно базе данных - специалисты связанные с работой с данными без опыта могут расчитывать на з/п в 74 тыс. руб, с опытом от 1 до 3 лет - на 140 тыс., с опытом от 3 до 6 лет - на 240 тыс.руб. 

Количество требуемых навыков в вакансиях связанных с данными не сильно отличается от количества навыков в вакансия в среднем. Так, в среднем по базе данных от соискателя ожидают владение 6.37 навыками, в то время как от специалистов по работе с данными - 6.41 навыками.


# Общий вывод по проекту

In [54]:
# Загрзузим данные в датафрэйм и сделаем пару наглядных графиков.
querry_vac_data = f"""
select v.name vac_name, v.key_skills, v.schedule, v.experience, v.employment, v.salary_from, v.salary_to, e.name emp_name, a.name vac_area, ar.name empl_area, v.employer_id empl_id
from public.vacancies v
left join public.employers e on v.employer_id = e.id
left join public.areas a on v.area_id = a.id
left join public.areas ar on ar.id = e.area
"""
df_vac_data = pd.read_sql_query(querry_vac_data,connection)

querry_empl_data = f"""
select e.name empl_name, i.name industry, e.id empl_id
from public.employers e
left join public.employers_industries ei on e.id = ei.employer_id
left join public.industries i on i.id = ei.industry_id
"""

df_empl_data = pd.read_sql_query(querry_empl_data,connection)


C:\Users\e.kuryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\e.kuryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [55]:
df_vac_data['mid_salary'] = np.where((df_vac_data.salary_from > 0)&(df_vac_data.salary_to > 0), 
(df_vac_data.salary_from+df_vac_data.salary_to)/2,df_vac_data.salary_from.replace(np.nan,0) +df_vac_data.salary_to.replace(np.nan,0))

df_vac_data.mid_salary = df_vac_data.mid_salary.replace(0,np.nan)

In [90]:
import plotly.express as px
fig = px.histogram(df_vac_data, x= 'mid_salary', color='experience',
                   title='Распределение зарплат в зависимости от опыта работы',
                   labels={'mid_salary':"Средняя указанная в вакансии зарплата, руб.",'value':'Количество вакансий'}).update_layout(yaxis_title = 'Количество вакансий')
                   #marginal="rug", # can be `box`, `violin`
                    #     hover_data=df.columns)
fig.show()
fig.write_html("Plotly/Mid_salary_distr.html")

In [64]:
top_15_vac = df_vac_data.vac_name.value_counts()[:15]
fig = px.bar(top_15_vac,labels={'index':'Название вакансии','value':'Количество вакансия'}, title= 'Топ 15 искомых специалистов')
fig.update_layout(showlegend = False)
fig.show()
fig.write_html("Plotly/top_vacancy_bar.html")


In [84]:
top_12_vac_areas = df_vac_data[df_vac_data.mid_salary>0].vac_area.value_counts().index[:12]
fig = px.box(df_vac_data[df_vac_data.vac_area.isin(top_12_vac_areas)], 
                                                   x = 'mid_salary', 
                                                   y = 'vac_area',
                                                   labels={'mid_salary':'Указанная в вакансии средняя з/п','vac_area':'Город'}, 
                                                   title= 'Ожидаемая средняя зарплата для топ 12 городов по вакансиям с указанной з/п. руб.')
fig.show()
fig.write_html('Plotly/expected_salary_city_box.html')

### Общий вывод

При поверхностном взгляде на данные можно сделать следующие выводы:
1. В представленной базе 49197 вакансий от 23501 работодателя, располагающихся в 1362 городах и разделенных на 294 сферы деятельности. 
2. В среднем на одного работодателя приходиться 2 вакансии. 
3. В среднем в одном городе располагается 17 работодателей и открыто 36 вакансий.
4. Среднее значения по количеству работодателей в каждой сфере деятельности 79.

Так как средние значения не дают корректной информации по данным, продолжили исследовать данные и получили следующие выводы:
1. Наибольшее число вакансий представлено в Москве - практически 11% от всех вакансий. 
2. На 12 городов (из 769) приходится 48% всех вакансий (23616 из 49197). 
3. Большинство вакансий открыто на полную занятость (71%) при этом в базе присутствует 15% вакансий с удаленной занятостью на полный рабочий день.
4. 51% открытых вакансий требуют опыта работы от 1 до 3 лет, с требование к опыту более 6 лет открыто всего 2.7% вакансий
5. В половине вакансий указана предлагаемая зарплата (в 24073 из 49197) при этом исходя из данных в базе видна прямая зависимость между опытом работы и предлагаемой зарплатной вилкой (что логично). Так для вакансий без опыта работы вилка в среднем составляет от 34699 до 56990, для вакансий с опытом работы более 6 лет вилка в среднем составляет от 150156 до 197819.
6. Наибольшую среднюю з/п (1 млн руб.) среди вакансий с указанными зарплатами предлагают в вакансии расположенной в Санкт-Петербурге. Для Москвы наибольшее значения составляет 750 тыс.руб.

Наибольшее количество работодателей находится в Москве, так же в этом городе открыто наибольшее количество вакансий. Собрав отдельно информацию по вакансиям "Яндекса", получили ожидаемую информацию о том, что наибольшее количество вакансий открыто так же в Москве.

Отдельно посмотрев на вакансии, связанные с обработкой данных, получили следующие значения:
1. В базе данных 3553 компании предлагаю вакансии связанные с ИТ. При этом 8419 работодателей не указали сферу деятельности.
2. Наиболее востребована вакансия Системного администратора - 1771 вакансия, следом идет "Программист 1с" - 1153 вакансии.
3. В базе данных 1771 вакансия каким-либо образом связанные с данными. При этом для джунов есть 51 вакансия, 201 вакансия связана со знание SQL, 351 вакансия связана со знанием Python.
4. Согласно базе данных - специалисты связанные с работой с данными без опыта могут рассчитывать на з/п в 74 тыс. руб, с опытом от 1 до 3 лет - на 140 тыс., с опытом от 3 до 6 лет - на 240 тыс.руб. 
5. Количество требуемых навыков в вакансиях, связанных с данными не сильно отличается от количества навыков в вакансия в среднем. Так, в среднем по базе данных от соискателя ожидают владение 6.37 навыками, в то время как от специалистов по работе с данными - 6.41 навыками.

